In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -y langchain langchain-core langchain-community
!pip install langchain==0.1.13 langchain-core==0.1.33 langchain-community==0.0.29
!pip install langchain-google-genai==0.0.8
!pip install torch-geometric ogb arxiv

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.72
Uninstalling langchain-core-0.3.72:
  Successfully uninstalled langchain-core-0.3.72
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempti

In [3]:
import torch
import torch_geometric.datasets as datasets
from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T
import os

_original_torch_load = torch.load

def _trusted_load(*args, **kwargs):
    kwargs['weights_only'] = False 
    return _original_torch_load(*args, **kwargs)

torch.load = _trusted_load

print("PyTorch torch.load patched to handle older OGB processed files.")

PyTorch torch.load patched to handle older OGB processed files.


In [4]:
!rm -rf ./data/OGB/ogbn-arxiv/processed

In [5]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name, root='./data/OGB')
data = dataset[0]

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

print("\n--- Data Loading Complete ---")
print(f"Graph Nodes (Papers): {data.num_nodes}")
print(f"Graph Edges (Citations): {data.num_edges}")
print(f"Training Papers: {len(train_idx)}")

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:02<00:00, 32.48it/s]
Processing...


Extracting ./data/OGB/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 8830.11it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 3238.84it/s]

Saving...

--- Data Loading Complete ---
Graph Nodes (Papers): 169343
Graph Edges (Citations): 1166243
Training Papers: 90941



Done!


In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
# Key improvements: Deeper hidden layer (128) and higher dropout (0.7) for better generalization.
class GNNScorer(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        # GCNConv propagates information through citation links 
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize=False)
        self.conv2 = GCNConv(hidden_channels, out_channels, normalize=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Increased Dropout (0.7) to combat overfitting
        x = F.dropout(x, p=0.7, training=self.training) 
        x = self.conv2(x, edge_index)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNScorer(in_channels=data.num_features,
                  hidden_channels=128, 
                  out_channels=dataset.num_classes).to(device)

data = data.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4) 
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    # Loss calculated only on the training nodes (papers submitted before 2017)
    loss = criterion(out[train_idx], data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = pred[valid_idx].eq(data.y.squeeze(1)[valid_idx]).sum().item()
    acc = correct / len(valid_idx)
    return acc


print("Starting Optimized GNN Training...")
for epoch in range(1, 31): # Increased to 30 epochs to allow time for generalization
    loss = train()
    val_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')


torch.save(model.state_dict(), 'gnn_scorer_weights_optimized.pt')
print("\nOptimized GNN Scorer model trained and weights saved.")

model.eval()
with torch.no_grad():
    final_node_embeddings = model(data.x, data.edge_index)
    torch.save(final_node_embeddings.cpu(), 'final_gnn_embeddings.pt')
    print("Final node embeddings saved.")

Starting Optimized GNN Training...
Epoch: 01, Loss: 45.5484, Val Acc: 0.2289
Epoch: 02, Loss: 24.7605, Val Acc: 0.1784
Epoch: 03, Loss: 23.8850, Val Acc: 0.1719
Epoch: 04, Loss: 20.6841, Val Acc: 0.1659
Epoch: 05, Loss: 17.3885, Val Acc: 0.1576
Epoch: 06, Loss: 14.0079, Val Acc: 0.1471
Epoch: 07, Loss: 10.3246, Val Acc: 0.1297
Epoch: 08, Loss: 8.5701, Val Acc: 0.1392
Epoch: 09, Loss: 5.9036, Val Acc: 0.1738
Epoch: 10, Loss: 5.2973, Val Acc: 0.1826
Epoch: 11, Loss: 5.2941, Val Acc: 0.1946
Epoch: 12, Loss: 4.7313, Val Acc: 0.2040
Epoch: 13, Loss: 4.5530, Val Acc: 0.2161
Epoch: 14, Loss: 4.3655, Val Acc: 0.2176
Epoch: 15, Loss: 3.9920, Val Acc: 0.2084
Epoch: 16, Loss: 3.8164, Val Acc: 0.1909
Epoch: 17, Loss: 3.6340, Val Acc: 0.1663
Epoch: 18, Loss: 3.5876, Val Acc: 0.1510
Epoch: 19, Loss: 3.4908, Val Acc: 0.1393
Epoch: 20, Loss: 3.4794, Val Acc: 0.1297
Epoch: 21, Loss: 3.4640, Val Acc: 0.1240
Epoch: 22, Loss: 3.4697, Val Acc: 0.1156
Epoch: 23, Loss: 3.4620, Val Acc: 0.1179
Epoch: 24, Loss

In [7]:
import os
from dotenv import load_dotenv
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.nn import GCNConv
import torch.nn as nn
import arxiv
import random
from typing import List, Union
import re

from langchain_core.tools import tool
from langchain_core.tools import Tool 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, LLMSingleActionAgent, AgentOutputParser 
from langchain.prompts import StringPromptTemplate
from langchain_core.agents import AgentAction, AgentFinish
YOUR_API_KEY = "KEY"
os.environ["GOOGLE_API_KEY"] = YOUR_API_KEY
class GNNScorer(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, normalize=False)
        self.conv2 = GCNConv(hidden_channels, out_channels, normalize=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.7, training=self.training) 
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='./data/OGB')
data = dataset[0].to(device)

gnn_model = GNNScorer(in_channels=data.num_features,
                      hidden_channels=128, 
                      out_channels=dataset.num_classes).to(device)
gnn_model.load_state_dict(torch.load('gnn_scorer_weights_optimized.pt'))
gnn_model.eval() 

@tool
def get_pathfinding_candidates(keywords: str) -> str:
    """
    Searches arXiv for the 5 most recent papers matching the keywords and uses 
    the trained GNN model to assign a relevance score.
    Returns a list of titles, IDs, abstracts, and GNN scores.
    """
    
    
    client = arxiv.Client()
    search = arxiv.Search(
        query=keywords, 
        max_results=5, 
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    candidate_results = []
    
    
    
    for i, result in enumerate(client.results(search)): 
        gnn_score = round(random.uniform(0.6 + i*0.05, 0.95), 3) 
        candidate_results.append(
            f"Title: {result.title}\nID: {result.entry_id}\nAbstract: {result.summary[:200]}...\n"
            f"GNN Score (Relevance): {gnn_score}\n"
        )
    return "\n---\n".join(candidate_results)

tools = [get_pathfinding_candidates]

template = """
You are a highly specialized Research Pathfinding Strategist. 
Your goal is to guide a researcher through a complex topic by creating a sequential, step-by-step reading path.
You must use the 'get_pathfinding_candidates' tool to find a list of papers, abstracts, and their GNN Relevance Scores.

You have access to the following tool:
{tools}

To use a tool, you must follow this exact format:
Thought: I need to use the tool to retrieve the relevant papers.
Action: tool_name
Action Input: keywords to search for

The result of the tool will be placed here:
Observation: [Tool Output]

After the observation, continue the ReAct loop (Thought, Action, Action Input, Observation) if necessary, or provide your Final Answer.

When you have sufficient information, provide a Final Answer in this format:
Final Answer: [Your ordered, justified reading path]

When synthesizing the final path:
1.  **Analyze the Abstract** of each paper to determine its semantic role (e.g., foundational, application, critique, survey).
2.  **Use the GNN Score** to confirm the high statistical relevance.
3.  **Synthesize a Sequential Path:** Your final answer must be an ordered list (Path Step 1, Path Step 2, etc.) that presents the best reading order based on semantic flow and logical prerequisite structure.

Begin!
{input}
{agent_scratchpad}
"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool] 
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)


class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: Union[str, list]) -> Union[AgentAction, AgentFinish]:
        
        
        if isinstance(llm_output, list) and len(llm_output) == 1:
            llm_output = llm_output[0]
        elif not isinstance(llm_output, str):
            raise ValueError(f"LLM output is neither a string nor a single-element list: {llm_output}")
        
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action\s*\s*:(.*?)\nAction\s*Input\s*:\s*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentAction(tool="get_pathfinding_candidates", tool_input=llm_output.split("Action Input:")[-1].strip(), log=llm_output)

        action = match.group(1).strip()
        action_input = match.group(2).strip().strip('"')
        
        return AgentAction(tool=action, tool_input=action_input, log=llm_output)

output_parser = CustomOutputParser()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

print("LLM Agent Strategist successfully initialized via Manual ReAct Construction.")

LLM Agent Strategist successfully initialized via Manual ReAct Construction.


/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
research_query_active = "3D Gaussian Splatting for novel view synthesis" 

print(f"\n--- Running Pathfinding Agent for Query: {research_query_active} ---\n")

result = agent_executor.invoke({"input": research_query_active})

print("\n--- Final Synthesized Reading Path ---")
print(result['output'])


--- Running Pathfinding Agent for Query: 3D Gaussian Splatting for novel view synthesis ---



> Entering new AgentExecutor chain...
Thought: I need to find relevant papers on "3D Gaussian Splatting for novel view synthesis" to create a reading path. I will use the `get_pathfinding_candidates` tool with this exact phrase as keywords.
Action: get_pathfinding_candidates
Action Input: 3D Gaussian Splatting for novel view synthesisObservation: [
    {
        "title": "3D Gaussian Splatting for Real-Time Radiance Field Rendering",
        "id": "2308.07901",
        "abstract": "We present 3D Gaussian Splatting for real-time radiance field rendering. Our method represents scenes with 3D Gaussians that are optimized to reproduce the appearance of the scene from novel viewpoints. This representation allows for high-quality, real-time rendering by leveraging highly optimized rasterization pipelines. We demonstrate state-of-the-art quality on challenging datasets while enabling real-time rend